   ## Import libraries

In [ ]:
import sys
import os

# Import local libs
module_path = os.path.abspath(os.path.join('..', 'libs'))
if module_path not in sys.path:
    sys.path.append(module_path)
from s3utils import *
from rooftop import *

# %load_ext autoreload
# %autoreload 2

   ## 1. Set variables and parameters

In [ ]:
# Paths and file names
bucket_name = 'roof-index'
main_dir = 'missoula/geospatial/'  # Main directory to read and write from in S3 bucket.
dsm_fname = 'downtown_dsm.tif'
hfdem_fname = 'downtown_hfdem.tif'
bldgs_fname = 'downtown_bldgs.zip'
bldgs_id = 'fid'  # Name of the unique id used in the buildings shapefile 
slope_fname = 'test_slope.tif'  # Does not exist. This is the name of the file to be written.
height_fname = 'test_height.tif'  # Does not exist. This is the name of the file to be written.
flat_area_fname = 'test_flat_area_bldgs.zip'  # Does not exist. This the name of the file to be written.

# Parameters
pitch_slope_threshold = 11  # Max. slope (degrees) to which we consider roof to be flat.
pitch_area_threshold = 9  # Min. percentage of flat area on a roof to be classified as flat.
fa_slope_threshold = 45  # Max. value of slope for flat area disaggregation
fa_area_threshold = 1000  # Min. area in s.f. for flat area disaggregation


   ## 3. Instantiate S3 and Rooftop objects

In [ ]:
S3 = S3Helper(bucket_name)
Rooftop = RooftopProc(S3, main_dir, dsm_fname, hfdem_fname, bldgs_fname, bldgs_id)

   ## 4. Create slope and height arrays and upload to S3 as geotiffs

In [ ]:
slope_arr = Rooftop.create_slope_arr(slope_fname) 
height_arr = Rooftop.create_height_arr(height_fname)

   ## 5. Filter out pitched roofs from bldgs dataframe

In [ ]:
flat_bldgs = Rooftop.pitched_roof_filter(pitch_area_threshold, pitch_area_threshold)

   ## 6. Disaggregate building footprints into flat areas

In [ ]:
disagg_bldgs = Rooftop.flat_area_disaggregator(flat_bldgs, fa_slope_threshold, fa_area_threshold, flat_area_fname)

   ## 7. Add rooftop features
 Features list below can include:

 - 'average_slope'
 - 'height'
 - 'parapet'
 - 'closeness_to_points'
 - 'volume_on_roof'

In [ ]:
features = ['average_slope', 'height', 'parapet', 'closeness_to_pts', 'volume_on_roof']
ctp_paths = ['test_greenspaces.zip', 'test_groceries.zip']
full_features = Rooftop.feature_builder(disagg_bldgs, features, ctp_paths=ctp_paths)

   ## 8. Create index